In [38]:
#Importing dependancies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [39]:
#Importing Sklearn
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [40]:
#Loading the cleaned Data
file_path = Path('highered_merged_data.csv')
df = pd.read_csv(file_path)
df.head(5)



c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Year,UNITID,FACSTAT,ARANK,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
0,2019,100654,0,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
1,2019,100654,10,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
2,2019,100654,10,1,49,37,12,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
3,2019,100654,10,2,50,34,16,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
4,2019,100654,10,3,98,46,52,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368


In [41]:
# df_test = df.drop_duplicates()
df_test = df.drop(['FACSTAT', 'ARANK'], axis=1)
df_test.sample(10)

,Year,UNITID,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
24630,2019,172477,3,1,2,Western Michigan University-Thomas M. Cooley L...,Lansing,MI,48933,2,9,2,-84.554429,42.731000
58178,2019,376385,17,10,7,Universal Technology College of Puerto Rico,Aguadilla,PR,603,2,5,1,-67.155628,18.428120
604,2019,101736,82,36,46,Northwest-Shoals Community College,Muscle Shoals,AL,35661,1,3,2,-87.678089,34.739788
55788,2019,240444,535,286,249,University of Wisconsin-Madison,Madison,WI,53706-1380,1,9,5,-89.404098,43.075409
60167,2019,440794,5,2,3,Pillar College,Newark,NJ,7102,2,7,1,-74.169153,40.738539
76509,2020,141565,12,1,11,University of Hawaii at Hilo,Hilo,HI,96720,1,9,2,-155.080847,19.701189
86810,2020,166054,7,5,2,Hellenic College-Holy Cross Greek Orthodox Sch...,Brookline,MA,2445,2,7,1,-71.129418,42.317018
30359,2019,187596,85,56,29,Navajo Technical University,Crownpoint,NM,87313-0849,1,7,2,-108.149392,35.687431
96322,2020,190503,33,23,10,Culinary Institute of America,Hyde Park,NY,12538,2,7,2,-73.933231,41.745806
55078,2019,239017,68,38,30,Lawrence University,Appleton,WI,54911,2,5,2,-88.397822,44.261323


In [42]:
df_test.drop_duplicates(inplace=True)

In [43]:
df_test1 = df_test.groupby(['UNITID', 'Year'])['HRTOTLT'].sum().unstack(fill_value=0)
#.unstack(fill_value=0)
df_test1.head(10)

Year,2019,2020,2021
UNITID,,,
100654,874,1176,1027
100663,9874,9961,10208
100690,117,100,120
100706,1578,1514,1534
100724,1026,996,970
100751,6746,6708,6928
100760,137,127,119
100812,274,256,292
100830,865,1011,1004


In [44]:
df1 = df.drop(["Year_y"], axis=1)
df1.sample(10)


KeyError: "['Year_y'] not found in axis"

In [ ]:
df2 = df1.rename(columns={"Year_x": "Year"})
df2.sample(5)

,Year,UNITID,FACSTAT,ARANK,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
8233,2021,131399,44,1,2,1,1,University of the District of Columbia,Washington,DC,20008,1,9,2,-77.066247,38.943819
58919,2021,441025,0,0,8,5,3,Miller-Motte College-Charleston,Charleston,SC,29418,3,3,1,-80.044585,32.953274
42827,2021,213774,40,3,9,2,7,Manor College,Jenkintown,PA,19046,2,6,1,-75.104786,40.087140
17963,2021,157085,20,1,621,453,168,University of Kentucky,Lexington,KY,40506,1,9,5,-84.504747,38.038911
43695,2021,215798,30,2,1,1,0,Saint Vincent College,Latrobe,PA,15650,2,9,2,-79.403762,40.292650


In [ ]:
#Creating feature
X = df2.drop("HRTOTLT", axis=1)
X = pd.get_dummies(X)

In [ ]:
#Adding a target thought? 
target = ["HRTOTLT"]

In [ ]:
#Creating a target
y = df1.loc[:, target].copy()

In [ ]:
#Decribing
X.describe()

,Year,UNITID,FACSTAT,ARANK,HRTOTLM,HRTOTLW,ZIP,CONTROL,HLOFFER,INSTSIZE,...,STABBR_TN,STABBR_TX,STABBR_UT,STABBR_VA,STABBR_VI,STABBR_VT,STABBR_WA,STABBR_WI,STABBR_WV,STABBR_WY
count,63625.0,63625.000000,63625.000000,63625.000000,63625.000000,63625.000000,63625.000000,63625.000000,63625.000000,63625.000000,...,63625.000000,63625.000000,63625.000000,63625.000000,63625.000000,63625.000000,63625.000000,63625.000000,63625.000000,63625.000000
mean,2021.0,208733.552692,28.712283,1.979976,35.397171,34.874986,46526.024094,1.610829,6.862570,2.396306,...,0.024031,0.057556,0.005910,0.029438,0.000534,0.004259,0.016613,0.018593,0.011033,0.002326
std,0.0,91957.196986,14.920225,1.825141,113.225757,99.260466,29723.392802,0.618448,2.326591,1.198726,...,0.153148,0.232904,0.076647,0.169032,0.023111,0.065125,0.127817,0.135085,0.104460,0.048174
min,2021.0,100654.000000,0.000000,0.000000,0.000000,0.000000,603.000000,1.000000,3.000000,-2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2021.0,153384.000000,10.000000,0.000000,2.000000,2.000000,20686.000000,1.000000,5.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2021.0,191533.000000,40.000000,2.000000,8.000000,10.000000,44115.000000,2.000000,7.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2021.0,223922.000000,42.000000,3.000000,27.000000,31.000000,72501.000000,2.000000,9.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2021.0,497286.000000,50.000000,6.000000,3855.000000,2828.000000,99801.000000,3.000000,9.000000,5.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(Counter(y_train['HRTOTLT']))
print(Counter(y_test['HRTOTLT']))

Counter({1: 4273, 2: 2597, 3: 2091, 4: 1747, 5: 1491, 6: 1363, 7: 1351, 8: 1264, 9: 1012, 10: 985, 11: 896, 12: 831, 13: 824, 15: 811, 14: 791, 17: 663, 16: 656, 18: 585, 19: 571, 20: 568, 26: 512, 21: 495, 25: 451, 24: 440, 23: 436, 22: 423, 29: 397, 27: 395, 31: 384, 28: 373, 32: 327, 34: 326, 30: 324, 35: 296, 33: 290, 36: 278, 39: 274, 37: 268, 43: 262, 38: 245, 41: 242, 44: 235, 42: 229, 40: 216, 48: 215, 57: 206, 45: 202, 49: 201, 53: 199, 46: 198, 47: 198, 54: 184, 56: 177, 51: 176, 64: 172, 52: 168, 50: 164, 62: 164, 55: 160, 60: 158, 67: 147, 69: 146, 68: 144, 61: 142, 63: 136, 58: 133, 59: 130, 66: 129, 75: 124, 71: 123, 77: 120, 65: 119, 80: 114, 81: 102, 86: 99, 85: 97, 79: 95, 78: 95, 87: 95, 76: 95, 73: 93, 72: 92, 88: 92, 92: 91, 91: 90, 74: 90, 90: 88, 84: 87, 70: 86, 113: 83, 101: 83, 93: 79, 96: 78, 98: 77, 121: 74, 105: 74, 125: 73, 97: 72, 83: 71, 109: 71, 127: 71, 82: 70, 99: 67, 137: 67, 102: 67, 95: 65, 118: 65, 128: 64, 108: 62, 94: 62, 131: 62, 111: 62, 115: 61

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_train, y_train)
print(Counter(y_train['HRTOTLT']))

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Counter({1: 4273, 2: 2597, 3: 2091, 4: 1747, 5: 1491, 6: 1363, 7: 1351, 8: 1264, 9: 1012, 10: 985, 11: 896, 12: 831, 13: 824, 15: 811, 14: 791, 17: 663, 16: 656, 18: 585, 19: 571, 20: 568, 26: 512, 21: 495, 25: 451, 24: 440, 23: 436, 22: 423, 29: 397, 27: 395, 31: 384, 28: 373, 32: 327, 34: 326, 30: 324, 35: 296, 33: 290, 36: 278, 39: 274, 37: 268, 43: 262, 38: 245, 41: 242, 44: 235, 42: 229, 40: 216, 48: 215, 57: 206, 45: 202, 49: 201, 53: 199, 46: 198, 47: 198, 54: 184, 56: 177, 51: 176, 64: 172, 52: 168, 50: 164, 62: 164, 55: 160, 60: 158, 67: 147, 69: 146, 68: 144, 61: 142, 63: 136, 58: 133, 59: 130, 66: 129, 75: 124, 71: 123, 77: 120, 65: 119, 80: 114, 81: 102, 86: 99, 85: 97, 79: 95, 78: 95, 87: 95, 76: 95, 73: 93, 72: 92, 88: 92, 92: 91, 91: 90, 74: 90, 90: 88, 84: 87, 70: 86, 113: 83, 101: 83, 93: 79, 96: 78, 98: 77, 121: 74, 105: 74, 125: 73, 97: 72, 83: 71, 109: 71, 127: 71, 82: 70, 99: 67, 137: 67, 102: 67, 95: 65, 118: 65, 128: 64, 108: 62, 94: 62, 131: 62, 111: 62, 115: 61

In [ ]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


0.23666590443081667

In [ ]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[867, 251,   0, ...,   0,   0,   0],
       [432, 212,   5, ...,   0,   0,   0],
       [254, 166,   5, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   1,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=int64)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.38      0.62      0.90      0.47      0.75      0.55      1396
          2       0.25      0.24      0.96      0.24      0.48      0.21       892
          3       0.08      0.01      1.00      0.01      0.09      0.01       632
          4       0.20      0.03      1.00      0.05      0.18      0.03       608
          5       0.10      0.06      0.98      0.07      0.24      0.05       455
          6       0.07      0.00      1.00      0.00      0.05      0.00       431
          7       0.08      0.03      0.99      0.04      0.17      0.02       472
          8       0.11      0.05      0.99      0.07      0.22      0.04       364
          9       0.08      0.12      0.97      0.09      0.35      0.11       346
         10       0.14      0.04      1.00      0.06      0.20      0.04       312
         11       0.09      0.01      1.00      0.02      0.10      0.01       292
   

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Sensitivity is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ee_model.fit(X_train, y_train)
print(Counter(y_train['HRTOTLT']))

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\ensemble\_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Counter({1: 4273, 2: 2597, 3: 2091, 4: 1747, 5: 1491, 6: 1363, 7: 1351, 8: 1264, 9: 1012, 10: 985, 11: 896, 12: 831, 13: 824, 15: 811, 14: 791, 17: 663, 16: 656, 18: 585, 19: 571, 20: 568, 26: 512, 21: 495, 25: 451, 24: 440, 23: 436, 22: 423, 29: 397, 27: 395, 31: 384, 28: 373, 32: 327, 34: 326, 30: 324, 35: 296, 33: 290, 36: 278, 39: 274, 37: 268, 43: 262, 38: 245, 41: 242, 44: 235, 42: 229, 40: 216, 48: 215, 57: 206, 45: 202, 49: 201, 53: 199, 46: 198, 47: 198, 54: 184, 56: 177, 51: 176, 64: 172, 52: 168, 50: 164, 62: 164, 55: 160, 60: 158, 67: 147, 69: 146, 68: 144, 61: 142, 63: 136, 58: 133, 59: 130, 66: 129, 75: 124, 71: 123, 77: 120, 65: 119, 80: 114, 81: 102, 86: 99, 85: 97, 79: 95, 78: 95, 87: 95, 76: 95, 73: 93, 72: 92, 88: 92, 92: 91, 91: 90, 74: 90, 90: 88, 84: 87, 70: 86, 113: 83, 101: 83, 93: 79, 96: 78, 98: 77, 121: 74, 105: 74, 125: 73, 97: 72, 83: 71, 109: 71, 127: 71, 82: 70, 99: 67, 137: 67, 102: 67, 95: 65, 118: 65, 128: 64, 108: 62, 94: 62, 131: 62, 111: 62, 115: 61

In [ ]:
# Calculated the balanced accuracy score
y_pred = ee_model.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


0.07169811320754717